In [1]:
import pandas as pd
import numpy as np


## Features engineering de pageviews

### Carga de datos

In [2]:
data = pd.read_pickle("../data/pageviews_total_formateado.pkl")

---

### Funciones auxiliares

#### Creacion de ventanas iniciales

In [3]:
def generar_dataframes_ventana(numero_ventana,dataframe):
    
    if numero_ventana == 4 :   #Para la ventana 4 se necesitan los 11 mil usuarios existentes
        df = pd.read_pickle("../data/users.pkl")
        df.set_index("USER_ID",inplace = True)
    
    else:    
        condicion = dataframe["trimestre"] == numero_ventana
        df = dataframe[condicion]["USER_ID"].drop_duplicates().to_frame().set_index("USER_ID").copy()

    return df.sort_index()
    

#### Generacion de features

In [4]:
def feature_ya_generado(nombre_feature, trimestre):
    """Corrobora si un feature ya fue generado """
    
    #Obtengo las columnas que genera ese feature
    columnas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
    for columna in columnas:
        if columna in FEATURES_YA_GENERADOS[trimestre]:
            return True
    return False

In [5]:
def generar_feature_en_ventana(dataframe,trimestre, nombre_feature, destino):
    """Genera un feature en la ventana correspondiente, con los tipos de datos correspondientes"""
    
    #Calculo el feature
    funcion_feature = FUNCIONES_FEATURES[nombre_feature]
    dataframe = funcion_feature( dataframe[dataframe["trimestre"] == trimestre] )

    #Uno los features nuevos con los viejos
    dataframe = destino[trimestre].merge(dataframe, left_index = True, right_index= True, how = 'left')
    
    #Rello los nans, si es que los hubiera
    dataframe = dar_formato_al_feature_nuevo(dataframe,nombre_feature)
    
    destino[trimestre] = dataframe

In [6]:
def dar_formato_al_feature_nuevo(dataframe, nombre_feature):
    """Rellena los Nans de un dataframe con features nuevos"""
    
    
    #Datos para rellenar los Nan
    columnas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
    tipo_dato= DATOS_FEATURES[nombre_feature][POS_TIPO_DATO]
    valor_nan = DATOS_FEATURES[nombre_feature][POS_DATO_NAN]
    
    dataframe.fillna(valor_nan, inplace = True)
    dataframe[columnas] = dataframe[columnas].astype(tipo_dato)
    
    return dataframe
    

In [7]:
def generar_features_faltantes(dataframe,destino,trimestre):
    """
    Funcion para generar los features faltantes para un trimestre, 
    evita la repeticion de calculo de features si estos ya se encuentran generados.
    
    dataframe: Dataframe a partir de cual se calcularan los features.
    destino: Diccionario de ventanas para los que calcular los features.
    trimestre: Numero de ventana a partir de la cual se calculan los features.
    """
    
    for nombre_feature in FUNCIONES_FEATURES:    
        if feature_ya_generado(nombre_feature, trimestre):
            continue

        #Si no fueron generados, los genero y actualizo el listado:   
        generar_feature_en_ventana(dataframe,trimestre, nombre_feature, destino)   
        
        #Agrego las columnas nuevas al diccionario
        columnas_generadas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
        FEATURES_YA_GENERADOS[trimestre].extend(columnas_generadas)

----

----

### Cargo features generados hasta el momento


In [8]:
# Trimestre 2 - 3 - 4 para los que tengo datos anteriores
CANT_TRIMESTRES = 4

In [9]:
TRIMESTRES = {}
trim_no_encontrados =[]

In [10]:
print("Cargando features")
for ventana_nro in range(1,CANT_TRIMESTRES) : #Trimestres del 1 al 3
    try:
        TRIMESTRES[ventana_nro] = pd.read_pickle("../data/features/ventana_{}_pageviews.pkl".format(ventana_nro))
        
    except FileNotFoundError:
        print("No se encontraron los features, se generarán los dataframe requeridos")
        trim_no_encontrados.append(ventana_nro)
    continue
    
for ventana_nro in trim_no_encontrados:    
    TRIMESTRES[ventana_nro] = generar_dataframes_ventana(ventana_nro,data)

Cargando features


### Set para predecir

In [11]:
try:
    print("Cargando features para predecir")
    TRIMESTRES[4] = pd.read_pickle("../data/features/predecir_pageviews.pkl")
except FileNotFoundError:
    print("No se encontro el feature deseado, se generara el dataframe requerido")
    TRIMESTRES[4] = generar_dataframes_ventana(4,data)

Cargando features para predecir


---

### Diccionario de features

In [12]:
#Posicion dentro del diccionario de features que ocupa la lista de columnas generadas por el mismo
POS_COLUMNAS_DEL_FEATURE = 0
POS_TIPO_DATO = 1
POS_DATO_NAN = 2

In [13]:
# Clave = nombre_feature : Valor = funcion
FUNCIONES_FEATURES = {}
# Clave = nombre_feature : Valor = (columnas,tipo_dato,valor_nan)
DATOS_FEATURES = {}

In [14]:
FEATURES_YA_GENERADOS = {}

for ventana_nro in range (1, CANT_TRIMESTRES + 1):
    
    FEATURES_YA_GENERADOS[ventana_nro] = list(TRIMESTRES[ventana_nro].columns.values)

---


---

## Creación de features

---

#### Vistazo del dataset: __Pageviews__

In [15]:
data.head()

,CONTENT_CATEGORY,CONTENT_CATEGORY_BOTTOM,CONTENT_CATEGORY_TOP,FEC_EVENT,ON_SITE_SEARCH_TERM,PAGE,SITE_ID,USER_ID,mes,trimestre
0,2,2,2,2018-01-01 00:09:17,1,2,2,7237,1,1
1,2,2,2,2018-01-01 00:14:17,1,2,2,4913,1,1
2,2,2,2,2018-01-01 00:14:18,1,2,2,4913,1,1
3,2,2,2,2018-01-01 00:14:26,1,3,3,4913,1,1
4,2,2,2,2018-01-01 00:14:32,1,20,3,4913,1,1


### Funciones auxiliares

#### Funcion para rellenar datos de un mes para el que no haya datos

In [16]:
def rellenar_columnas_multilevel_faltantes(dataframe,prefijo_columna, dato_nan):
    """prefijo_columna:   nombre de la columna de la que se calcularon los datos.
       dato_nan:          dato para rellenar los nan.
       inicio_multilevel: posicion(int) dentro de las columnas donde inicia la seccion a rellenar.
       Devuelve el nombre de las columnas en el orden correcto"""
    
    for mes in range(1,4):
        
        try:            #si no existe el mes, lo creo
            nombre_col = prefijo_columna+"{}".format(mes)
            dataframe[nombre_col] = dataframe[nombre_col].fillna(dato_nan)
  
        except KeyError:
            dataframe[nombre_col] = -1
            
    return [(prefijo_columna+"{}".format(mes)) for mes in range(1,4)]
    

---

#### Cantidad de paginas vista por un usuario en todo el trimestre

In [17]:
nombre = "cantidad paginas vistas"

In [18]:
def cantidad_de_paginas_vistas(dataframe):
    
    feature = dataframe.groupby("USER_ID").agg({"FEC_EVENT":"count"})\
                        .rename(columns = {"FEC_EVENT":"cantidad_paginas_vistas_trimestre"}) 
    return feature

In [19]:
FUNCIONES_FEATURES[nombre] = cantidad_de_paginas_vistas

In [20]:
columnas_generadas = ["cantidad_paginas_vistas_trimestre"] #lista
tipo_dato = "int64"
dato_nan = 0

DATOS_FEATURES[nombre] = (columnas_generadas, tipo_dato, dato_nan)

---

#### Cantidad de paginas vista en cada mes dentro del trimestre

In [21]:
nombre = "cantidad paginas vistas mes"

In [22]:
def cantidad_paginas_vistas_por_mes(dataframe):
    
    vistas_por_mes = dataframe.groupby(["USER_ID","mes"]).agg({"trimestre":"count"})
    vistas_por_mes.reset_index(inplace = True)
    vistas_por_mes = vistas_por_mes.pivot(index = "USER_ID", columns = "mes", values = "trimestre")
    
    vistas_por_mes.columns = ["cant_paginas_vistas_en_mes_{}".format(mes) for mes in range(1,4)]

    return vistas_por_mes

In [23]:
FUNCIONES_FEATURES[nombre] = cantidad_paginas_vistas_por_mes

In [24]:
columnas_generadas = ["cant_paginas_vistas_en_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int64"
dato_nan = 0

DATOS_FEATURES[nombre] = (columnas_generadas,tipo_dato,dato_nan) 

---

#### Page mas vista cada mes

In [25]:
nombre = "page mas vista mes"

In [26]:
def page_mas_vista_cada_mes(dataframe):
    
    feature = dataframe.groupby(['USER_ID','mes','PAGE']).agg({'trimestre':'count'})
    index = feature.groupby(['USER_ID','mes'])['trimestre'].transform(max) == feature['trimestre']
    feature = feature[index]
    
    #Elimino duplicados
    feature.reset_index(inplace=True)
    feature = feature.drop_duplicates(subset=['USER_ID','mes'])  
       
    
    #Cambio la forma del dataframe -> indice = USER_ID, columnas = "meses (1,2,3), valores = "pagina"
    feature = feature.pivot(index = "USER_ID",columns = "mes", values = ["PAGE"])
    
    feature.columns = [x for x in range(1,4)]
    feature.columns = [("page_top_en_mes_{}".format(mes)) for mes in feature.columns.values]
    
    
    nombres_col = rellenar_columnas_multilevel_faltantes(feature,"page_top_en_mes_",-1)
    
    return feature[nombres_col]

#### Funcion vieja

def page_mas_vista_cada_mes(dataframe):
    
    feature = dataframe.groupby(['USER_ID','mes','PAGE']).agg({'trimestre':'count'})
    index = feature.groupby(['USER_ID','mes'])['trimestre'].transform(max) == feature['trimestre']
    feature = feature[index]
    
    #Elimino duplicados
    feature.reset_index(inplace=True)
    feature = feature.drop_duplicates(subset=['USER_ID','mes'])  
    
    
    #Cambio la forma del dataframe -> indice = USER_ID, columnas = "meses (1,2,3), valores = "pagina"
    feature = feature.pivot(index = "USER_ID",columns = "mes", values = "PAGE")
        
    
    #lleno los meses vacios con -1
    feature.columns = [x for x in range(1,4)]
    for mes in range(1,4):
        try:            #si no existe el mes, lo creo
            
            feature[mes] = feature[mes].fillna(-1)
        except KeyError:
            feature.insert(mes,mes,-1)

                
    feature.columns = ["page_mas_vista_en_mes_{}".format(mes) for mes in range(1,4)]
    
    return feature

In [27]:
FUNCIONES_FEATURES[nombre] = page_mas_vista_cada_mes

In [28]:
columnas_generadas = ["page_top_en_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int16"
dato_nan = -1

DATOS_FEATURES[nombre]=(columnas_generadas, tipo_dato, dato_nan)

### PAGEs importantes vistas en cuatrimestre

In [29]:
nombre="PAGEs importantes vistas en cuatri"

In [30]:
PAGEs_importantes=[  41,  110,  109,  108,    2,  285,    3,  286,  345,   27,    5,
              57,  287,   23,   20,    4,   40,   21,   26,   65,   69,   42,
              32,   85,   58,   48,   99,   24,   61,   82,    1,   64,   81,
              59,  140,  296,   60,  182,    6,  153,   39,  146,   14,  186,
             187, 1259,   45,  161,  321,  369]  #Obtenidas mediante importancia de features de un Random Forest

In [31]:
def pages_importantes_vistas_cuatrimestre(dataframe):
    
    feature = pd.crosstab(dataframe.USER_ID,dataframe.PAGE)
    feature = feature.reindex(PAGEs_importantes).fillna(0).astype(int) 
       
    feature.columns = [("PAGE_{}".format(x)) for x in PAGEs_importantes]
    
    return feature

In [32]:
FUNCIONES_FEATURES[nombre] = pages_importantes_vistas_cuatrimestre

In [33]:
columnas_generadas = [("PAGE_{}".format(x)) for x in PAGEs_importantes]
tipo_dato = "int16"
dato_nan = 0

DATOS_FEATURES[nombre]=(columnas_generadas, tipo_dato, dato_nan)

#### Cantidad de vistas de la pagina top en cada mes

In [34]:
nombre = "vistas pagina top mes"

In [35]:
def vistas_pagina_top_por_mes(dataframe):
    
    feature = dataframe.groupby(['USER_ID','mes','PAGE']).agg({'trimestre':'count'})
    index = feature.groupby(['USER_ID','mes'])['trimestre'].transform(max) == feature['trimestre']
    feature = feature[index]
    
    #Elimino duplicados
    feature.reset_index(inplace=True)
    feature = feature.drop_duplicates(subset=['USER_ID','mes'])  
       
    
    #Cambio la forma del dataframe -> indice = USER_ID, columnas = "meses (1,2,3), valores = "pagina"
    feature = feature.pivot(index = "USER_ID",columns = "mes", values = ["trimestre"])
    
    feature.columns = [x for x in range(1,4)]
    feature.columns = [("vistas_page_top_en_mes_{}".format(mes)) for mes in feature.columns]
    
    
    nombres_col = rellenar_columnas_multilevel_faltantes(feature,"vistas_page_top_en_mes_",0)
    
    return feature

In [36]:
FUNCIONES_FEATURES[nombre] = vistas_pagina_top_por_mes

In [37]:
columnas_generadas = ["vistas_page_top_en_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int64"
dato_nan = 0

DATOS_FEATURES[nombre]=(columnas_generadas, tipo_dato, dato_nan)

---

### Sitio mas visitado cada mes

In [38]:
nombre = "sitio mas buscado cada mes"

In [39]:
def sitio_mas_visitado_cada_mes(dataframe):
    
    feature = dataframe.groupby(['USER_ID','mes','SITE_ID']).agg({'trimestre':'count'})
    index = feature.groupby(['USER_ID','mes'])['trimestre'].transform(max) == feature['trimestre']
    feature = feature[index]
    
    #Elimino duplicados
    feature.reset_index(inplace=True)
    feature = feature.drop_duplicates(subset=['USER_ID','mes'])  
       
    
    #Cambio la forma del dataframe -> indice = USER_ID, columnas = "meses (1,2,3), valores = "pagina"
    feature = feature.pivot(index = "USER_ID",columns = "mes", values = ["SITE_ID"])
    
    feature.columns = [x for x in range(1,4)]
    feature.columns = [("sitio_mas_visitado_en_mes_{}".format(mes)) for mes in feature.columns.values]
    nombres_col = rellenar_columnas_multilevel_faltantes(feature,"sitio_mas_visitado_en_mes_",-1)
    
    return feature[nombres_col]

In [40]:
FUNCIONES_FEATURES[nombre] = sitio_mas_visitado_cada_mes

In [41]:
columnas_generadas = ["sitio_mas_visitado_en_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int16"
dato_nan = -1

DATOS_FEATURES[nombre]=(columnas_generadas, tipo_dato, dato_nan)

#### Cantidad de vistas sitio mas buscado por mes

In [42]:
nombre = "vistas de sitio top por mes"

In [43]:
def cantidad_vistas_sitio_top_por_mes(dataframe):
    
    feature = dataframe.groupby(['USER_ID','mes','SITE_ID']).agg({'trimestre':'count'})
    index = feature.groupby(['USER_ID','mes'])['trimestre'].transform(max) == feature['trimestre']
    feature = feature[index]
    
    #Elimino duplicados
    feature.reset_index(inplace=True)
    feature = feature.drop_duplicates(subset=['USER_ID','mes'])  
       
    
    #Cambio la forma del dataframe -> indice = USER_ID, columnas = "meses (1,2,3), valores = "pagina"
    feature = feature.pivot(index = "USER_ID",columns = "mes", values = ["trimestre"])
    
    feature.columns = [x for x in range(1,4)]
    feature.columns = [("vistas_site_top_mes_{}".format(mes)) for mes in feature.columns.values]

    nombres_col = rellenar_columnas_multilevel_faltantes(feature,"vistas_site_top_mes_",0)
    
    return feature[nombres_col]

In [44]:
FUNCIONES_FEATURES[nombre] = cantidad_vistas_sitio_top_por_mes

In [45]:
columnas_generadas = ["vistas_site_top_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int64"
dato_nan = 0

DATOS_FEATURES[nombre]=(columnas_generadas, tipo_dato, dato_nan)

---

#### Termino mas buscado en cada mes

In [46]:
nombre = "termino mas buscado cada mes"

In [47]:
def termino_mas_buscado_cada_mes(dataframe):
    
    feature = dataframe.groupby(['USER_ID','mes','ON_SITE_SEARCH_TERM']).agg({'trimestre':'count'})
    index = feature.groupby(['USER_ID','mes'])['trimestre'].transform(max) == feature['trimestre']
    feature = feature[index]
    
    #Elimino duplicados
    feature.reset_index(inplace=True)
    feature = feature.drop_duplicates(subset=['USER_ID','mes'])  
       
    
    #Cambio la forma del dataframe -> indice = USER_ID, columnas = "meses (1,2,3), valores = "pagina"
    feature = feature.pivot(index = "USER_ID",columns = "mes", values = ["ON_SITE_SEARCH_TERM"])
    
    feature.columns = [x for x in range(1,4)]
    feature.columns = [("termino_top_en_mes_{}".format(mes)) for mes in feature.columns.values]
    
    
    nombres_col = rellenar_columnas_multilevel_faltantes(feature,"termino_top_en_mes_",-1)
    
    return feature[nombres_col]

In [48]:
FUNCIONES_FEATURES[nombre] = termino_mas_buscado_cada_mes

In [49]:
columnas_generadas = ["termino_top_en_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int16"
dato_nan = -1

DATOS_FEATURES[nombre]=(columnas_generadas, tipo_dato, dato_nan)

#### Cantidad de busquedas termino mas buscado por mes

In [50]:
nombre = "busquedas de termino top por mes"

In [51]:
def cantidad_busquedas_termino_top_por_mes(dataframe):
    
    feature = dataframe.groupby(['USER_ID','mes','ON_SITE_SEARCH_TERM']).agg({'trimestre':'count'})
    index = feature.groupby(['USER_ID','mes'])['trimestre'].transform(max) == feature['trimestre']
    feature = feature[index]
    
    #Elimino duplicados
    feature.reset_index(inplace=True)
    feature = feature.drop_duplicates(subset=['USER_ID','mes'])  
       
    
    #Cambio la forma del dataframe -> indice = USER_ID, columnas = "meses (1,2,3), valores = "pagina"
    feature = feature.pivot(index = "USER_ID",columns = "mes", values = ["trimestre"])
    
    feature.columns = [x for x in range(1,4)]
    feature.columns = [("busquedas_termino_top_mes_{}".format(mes)) for mes in feature.columns.values]
    
    
    nombres_col = rellenar_columnas_multilevel_faltantes(feature,"busquedas_termino_top_mes_",0)
    
    return feature[nombres_col]

In [52]:
FUNCIONES_FEATURES[nombre] = cantidad_busquedas_termino_top_por_mes

In [53]:
columnas_generadas = ["busquedas_termino_top_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int64"
dato_nan = 0

DATOS_FEATURES[nombre]=(columnas_generadas, tipo_dato, dato_nan)

---

---

## Generacion de features

In [54]:
for ventana_nro in range(1, CANT_TRIMESTRES + 1):
    generar_features_faltantes(data, TRIMESTRES, ventana_nro)

MemoryError: 

In [ ]:
TRIMESTRES[1].head()

In [ ]:
TRIMESTRES[4].head()

### Guardado

In [99]:
for ventana_nro in range(1, CANT_TRIMESTRES ):
    TRIMESTRES[ventana_nro].to_pickle("../data/features/ventana_{}_pageviews.pkl".format(ventana_nro))

TRIMESTRES[4].to_pickle("../data/features/predecir_pageviews.pkl")

---